#/ REQUIREMENTS #/

In [1]:
### Natural Language Processing & ML:
%pip install openai
%pip install langchain
%pip install tiktoken

### Data Manipulation & Extraction:
%pip install pypdf
%pip install chromadb

### Configuration & Environment:
%pip install python-dotenv

### Web Development & Server:
%pip install fastapi
%pip install nest-asyncio
%pip install pyngrok
%pip install uvicorn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#/ SETTING UP #/

In [1]:
### Natural Language Processing & ML:
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

### Basic Utilities:
import os

### Configuration & Environment:
from dotenv import load_dotenv, find_dotenv

### Web Development & Server:
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok

### Asynchronous Utilities:
import nest_asyncio


files_location = "./data/"
data = os.listdir(files_location)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

#/ LOAD_DATA #/

In [2]:
pdfsLoader = PyPDFDirectoryLoader(files_location)
documents = pdfsLoader.load()

#/ SPLITTER #/

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

texts = text_splitter.split_documents(documents)

#/ VECTOR_STORE #/

In [6]:
vector_store = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings(), persist_directory='./vector_store/')

vector_store.persist()

#/ COMUNICATION USING LLM :3 #/

In [7]:
chat_history = []
chat = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vector_store.as_retriever())

In [8]:
def chat_Prompt(question, chat_history):
    response = chat({"question": question,
                           "chat_history": chat_history})
    return response["answer"]

In [9]:
chat_Prompt("Que es la Cinematica y nombrame 3 temas", chat_history)

' La Cinemática es una parte de la Mecánica Clásica que estudia el movimiento de los cuerpos, sin explicar las causas que originan dicho movimiento. Los temas principales son: definición de Cinemática, elementos de Cinemática (como la partícula) y las leyes de movimiento de Newton.'

#/ FAST_API #/

In [10]:
app = FastAPI()

origins = ["null"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class Prompt(BaseModel):
    user_prompt : str

@app.post("/response/{Prompt}")
async def Post_prompt(prompt : Prompt):
    return {"response" : chat_Prompt(prompt.user_prompt, chat_history)}

nest_asyncio.apply()
uvicorn.run(app, port=8081)

INFO:     Started server process [90856]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8081 (Press CTRL+C to quit)


INFO:     127.0.0.1:58044 - "OPTIONS /response/Que%20es%20la%20Cinematica%20y%20nombrame%203%20temas HTTP/1.1" 200 OK
INFO:     127.0.0.1:58044 - "POST /response/Que%20es%20la%20Cinematica%20y%20nombrame%203%20temas HTTP/1.1" 200 OK
INFO:     127.0.0.1:58053 - "OPTIONS /response/Give%20me%20a%20equation%20for%20Newtons%20laws%20or%20movement HTTP/1.1" 200 OK
INFO:     127.0.0.1:58053 - "POST /response/Give%20me%20a%20equation%20for%20Newtons%20laws%20or%20movement HTTP/1.1" 200 OK
